# Student Loan Risk with Deep Learning

In [ ]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [ ]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [ ]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [ ]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [ ]:
# Define the target set y using the credit_ranking column
y = loans_df['credit_ranking']


# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [ ]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop(columns=['credit_ranking'])

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [ ]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [ ]:

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)

# Transform the features testing dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [ ]:
# Define the the number of inputs (features) to the model
num_inputs = X_train_scaled.shape[1]

# Review the number of features
print("Number of features:", num_inputs)

Number of features: 11


In [ ]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 64

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 32

# Define the number of neurons in the output layer
num_outputs = 1  # Assuming this is a binary classification task, otherwise adjust accordingly

In [ ]:
# Create the Sequential model instance
model = Sequential()

# Add the first hidden layer
model.add(Dense(hidden_nodes_layer1, input_shape=(num_inputs,), activation='relu'))

# Add the second hidden layer
model.add(Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(num_outputs, activation='sigmoid'))

In [ ]:
# Display the Sequential model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                768       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2881 (11.25 KB)
Trainable params: 2881 (11.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [ ]:
# Compile the Sequential model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model using 50 epochs and the training data
history = model.fit(X_train_scaled, y_train, epochs=50, verbose=1)

Epoch 1/50
40/40 [==============================] - 3s 8ms/step - loss: 0.6542 - accuracy: 0.6161
Epoch 2/50
40/40 [==============================] - 0s 6ms/step - loss: 0.5463 - accuracy: 0.7287
Epoch 3/50
40/40 [==============================] - 0s 7ms/step - loss: 0.5186 - accuracy: 0.7451
Epoch 4/50
40/40 [==============================] - 0s 6ms/step - loss: 0.5040 - accuracy: 0.7615
Epoch 5/50
40/40 [==============================] - 0s 12ms/step - loss: 0.4933 - accuracy: 0.7662
Epoch 6/50
40/40 [==============================] - 0s 9ms/step - loss: 0.4849 - accuracy: 0.7647
Epoch 7/50
40/40 [==============================] - 0s 8ms/step - loss: 0.4788 - accuracy: 0.7740
Epoch 8/50
40/40 [==============================] - 0s 8ms/step - loss: 0.4738 - accuracy: 0.7654
Epoch 9/50
40/40 [==============================] - 0s 8ms/step - loss: 0.4700 - accuracy: 0.7756
Epoch 10/50
40/40 [==============================] - 0s 9ms/step - loss: 0.4637 - accuracy: 0.7787
Epoch 11/50
40/40 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [ ]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Display the model loss and accuracy results
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

10/10 [==============================] - 0s 2ms/step - loss: 0.5349 - accuracy: 0.7500
Test Loss: 0.5348924398422241
Test Accuracy: 0.75


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [ ]:

# Set the model's file path
model_file_path = "student_loans.keras"

# Export your model to a keras file
model.save(model_file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [ ]:

# Set the model's file path
model_file_path = "student_loans.keras"

# Load the model to a new object
loaded_model = tf.keras.models.load_model(model_file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:

# Make predictions with the test data
predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(predictions[:5])

10/10 [==============================] - 0s 2ms/step
[[0.06178396]
 [0.42177802]
 [0.853871  ]
 [0.7070695 ]
 [0.97092116]]


In [ ]:

# Round the predictions to binary results (0 or 1)
binary_predictions = np.round(predictions).astype(int)

# Save the predictions to a DataFrame
predictions_df = pd.DataFrame(binary_predictions, columns=['Predicted_Credit_Ranking'])

# Display a sample of the predictions DataFrame
print(predictions_df.head())

   Predicted_Credit_Ranking
0                         0
1                         0
2                         1
3                         1
4                         1


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Convert predictions to binary labels
binary_labels = np.round(y_test).astype(int)

# Print the classification report
print(classification_report(binary_labels, binary_predictions))

              precision    recall  f1-score   support

           0       0.73      0.75      0.74       154
           1       0.77      0.75      0.76       166

    accuracy                           0.75       320
   macro avg       0.75      0.75      0.75       320
weighted avg       0.75      0.75      0.75       320



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.
  
  
    Personal informtion like name, age, gender, and employment status.
    loan information like loan amout, loan term, interst rate preferance, and type of loan.
    a list of possible loan options.
    relevent legal regulatory info.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.


     Content based filtering enables personalized recommendations by focusing on the specific characteristics.


3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.


    1. how i would maximize how much money i can milk from these debtors. this would be a challenge beccause data on typical term of a loan is very weighted data typially.

    2. making sure you give them the optimal loan reccomendation so you can full rake them over the coals. this would be hard because it requires haveing a very thorough and very big dataset which woud be hard to obtain.


**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

